# Introduction

The objective of the TensorSAT project is to build an SMT solver based on techniques analogous to [hyper-optimized tensor network contraction](https://arxiv.org/abs/2002.01935) techniques used in quantum simulation.

To achieve this, TensorSAT replaces predicates and formulae with categorical diagrams, taking their idealised semantics into the [hypergraph category](https://arxiv.org/abs/1806.08304) of sets and relations.
Functions and predicates are replaced by general relations, with synthetic composition rules used to contract diagrams and compute answers to satisfiability problems.

A **diagram** is formed by **wiring** together **boxes** from one or more **languages**, compatibly with the **types** specified by the boxes' **ports**.
Every language defines (parametrised families of) boxes and types, together with computational semantics in the form of contractions for arbitrary wirings of boxes in the language.
Such conctractions can be reduced to three special cases:

- `spider`: converts a wire &mdash; connecting any positive number of ports &mdash; into a box in the language.
- `transpose`: reorders the ports of any box in the language.
- `contract2`: contracts arbitrary wirings of any two boxes in the language (wired ports may be left open, but no copying is allowed).

Languages are allowed to overlap in boxes and types, and diagrams can involve boxes from multiple languages.

### A Categorical Aside

In categorical terms, each language is the hypergraph category freely generated by a parametrised family of types (object generators) and boxes (morphism generators).
Boxes are normalised to have the tensor unit as their domain: their signature takes the form $I \rightarrow X_0 \otimes ... \otimes X_{n-1}$, the indices $\lbrace 0, ..., n-1 \rbrace$ are the ports of the box, and we say that the generator $X_j$ is the type of port $j$.

A diagram involving boxes from one or more languages is a diagram in hypergraph category freely-generated by the union of their types and boxes.
The wiring of boxes in a diagram is a normalised representation of the diagram's compositional structure modulo the axioms of hypergraph categories (cf. [arXiv:1512.01602](https://arxiv.org/abs/1512.01602)).

Mathematically, languages have semantics in the hypergraph category Rel of sets and finitary relations.
Computationally, the semantics for each individual language are captured by providing an implementation for the `spider`, `transpose` and `contract2` rewrites which is sound for the Rel semantics (i.e. one such that the rewrite corresponds to an equation in Rel).

Formally, the rewrites equations turning a diagram in the language (two boxes, one box, and no boxes, respectively) into a single box in the same language.
Taken together, they give rise to a strongly confluent rewriting system which can be used to contract arbitrary diagrams in the language into a single box of the same language, providing computational semantics.

Types and boxes for multiple languages are allowed to overlap, and it is requried that the Rel semantics be compatible over such overlaps; that is, multiple languages assign the same set semantics to the same type, and the same relation semantics to the same box, whenever they have types and/or boxes in common.
Because of this, contraction in diagrams involving multiple languages does not necessarily result in a confluent rewriting system.

## 1. Languages


Types and boxes are the basic building blocks of a diagrammatic language.
Diagrams are formed by wiring boxes (or sub-diagrams) together, relying on matching types to determine which wirings are allowed and which ones are not.

Every language defines boxes and/or types, by means of concrete subclasses of the abstract base classes `Box` and `Type`, respectively.

```py
from tensorsat.diagrams import Box, Type
```

As our running example, we will use the language `fin_rel` of "finite relations":

- The types of `fin_rel` are instances of the class `FinSet`. They are finite, explicitly enumerated sets in the form $\lbrace 0,...,n-1 \rbrace$.
- The boxes of `fin_rel` are instances of the class `FinRel`. They are finite relations bewtween finite sets, explicitly present by dense Boolean tensors.

Languages are implemented as sub-modules of the `tensorsat.lang` module.

In [1]:
from tensorsat.lang.fin_rel import FinSet, FinRel

## Types

Types are used to signal compatibility of wiring between ports of boxes.

The constructors of concrete sub-classes of `Type` define the parametrisation for the types in the language. In the case of `FinSet`, explicitly enumerated sets are parametrised by a strictly positive integer size.

In [2]:
bit = FinSet(2)  # {0, 1}
trit = FinSet(3) # {0, 1, 2}

Types can be combined into Shapes &mdash; sequences of types &mdash; by "tensor" product:

In [3]:
s = bit * bit * trit
print(f"{s = }")
print(f"{len(s) = }")
print(f"{list(s) = }")
print(f"{s[2] = }")

s = <Shape 0x1c4e500a450: 3 components>
len(s) = 3
list(s) = [FinSet(2), FinSet(2), FinSet(3)]
s[2] = FinSet(3)


Iterated products of a type with itself can be constructed using the power operator:

In [4]:
kilobit = bit**1024
print(f"{kilobit = }")
print(f"{len(kilobit) = }")

kilobit = <Shape 0x1c4f48c1d90: 1024 components>
len(kilobit) = 1024


Types also define some utility methods to create special kinds of boxes. One such method defined on all types is `spider`, which creates the special box corresponding to the given number of copies of the type wired together. This implements the first of the three elementary rewrite rules for the language.

In [5]:
bit.spider(3)

<FinRel 0x1c4e63d3410: 3 ports>

## Boxes

Boxes act as the atomic building blocks of diagrams.

The constructors of the concrete subclass(es) of `Box` define the parametrisatino of boxes in the language. In the case of `FinRel`, relations are parametrised by Boolean tensors, in the form of NumPy's `uint8` arrays with values in $\lbrace 0, 1 \rbrace$.

In [ ]:
import numpy as np
and_ = FinRel(np.array([
    [
        [
            1 if out == inl & inr else 0
            for out in [0, 1]
        ]
        for inr in [0, 1]
    ]
    for inl in [0, 1]
], dtype=np.uint8))
and_

<FinRel 0x1c4f486ef90: 3 ports>

Because the raw parametrisation can be inconvenient, concrete subclasses of `Box` typically offer additional utility constructors, in the form of class methods. In the case of `FinRel`, examples include constructors from:

- explicit sets of tuples, via `FinRel.from_set`
- input-output mappings of functions, via `FinSet.from_mapping`
- implementations of functions, via `FinSet.from_callable`


In [7]:
assert and_ == FinRel.from_set((2,2,2), {
    #       input bits at start ^^^ ^ output bit at end
    (a, b, a&b)
    for a in [0, 1]
    for b in [0, 1]
})
assert and_ == FinRel.from_callable(
    (2, 2), # input shape (2 bits input)
    2,      # output shape (1 bit output), same as (2,)
    lambda a, b: a&b # function implementation
    #            ^^^ same as (a&b,)
)
assert and_ == FinRel.from_mapping((2,2), 2, {
    (a, b): a&b
    #       ^^^ same as (a&b,)
    for a in [0, 1]
    for b in [0, 1]
})

All boxes have a `shape`, which defines the number and type of ports for the box.

In [8]:
print(f"{and_.shape = }")
print(f"{list(and_.shape) = }")

and_.shape = <Shape 0x1c4f48c2310: 3 components>
list(and_.shape) = [FinSet(2), FinSet(2), FinSet(2)]


Each concrete `Box` subclass defines additional properties exposing the data parametrising them. In the case of `FinRel`, the `tensor` property exposes the underlying Boolean tensor (as a readonly view).

In [9]:
and_.tensor

array([[[1, 0],
        [1, 0]],

       [[1, 0],
        [0, 1]]], dtype=uint8)

Concrete `Box` subclasses also define properties and methods relevant to the language's semantics, especially relating boxes to various kinds of data which can be passed to utility constructors of the box class.

In [23]:
# Inverse of FinRel.from_set:
print(f"{set(and_.to_set()) = }")
# {(0, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 1)}

# Inverse of FinRel.from_mapping:
input_ports = [0, 1]
print(f"{and_.to_mapping(input_ports) = }")
# {(0, 0): (0,), (0, 1): (0,), (1, 0): (0,), (1, 1): (1,)}

# Inverse of FinRel.from_callable:
and_func = and_.to_callable(input_ports)
print(f"{and_func = }")
# <function <lambda> at 0x000001C4F48B1D00>
input_sizes = tuple(t.size for t in and_.shape[input_ports])
for i0, i1 in np.ndindex(input_sizes):
    print(f"and_func({i0}, {i1}) = {and_func(i0, i1)}")

set(and_.to_set()) = {(1, 0, 0), (0, 0, 0), (1, 1, 1), (0, 1, 0)}
and_.to_mapping(input_ports) = {(0, 0): (0,), (0, 1): (0,), (1, 0): (0,), (1, 1): (1,)}
and_func = <function <lambda> at 0x000001C4F48B2660>
and_func(0, 0) = (0,)
and_func(0, 1) = (0,)
and_func(1, 0) = (0,)
and_func(1, 1) = (1,)


## Libraries

Thematic collections of named types and boxes are made available as part of libraries, together with utility classes and functions. Libraries are implemented as sub-modules of the `tensorsat.lib` module.

For our subsequent examples, we will use types and boxes from the `bincirc` library, implementing binary circuits.

In [24]:
from tensorsat.lib.bincirc import (
    bit,   # bit type {0, 1}
    bit_0, # bit constant value 0
    bit_1, # bit constant value 1
    not_,  # unary NOT operator
    and_,  # binary AND operator
    or_,   # binary OR operator
    xor_,  # binary XOR operator
)

## Composition

Concrete `Box` subclasses also implement language-specific composition logic, which is accessible via two methods:

- the method `transpose`, for port permutation
- the class method `contract2`, for pairwise composition

We look at each one in turn, below.

The `transpose` method can be called on a box instance to produce another instance of the same box class &mdash; and, as a consequence, a box in the same language &mdash; where ports of the original box have been permuted as specified by the argument to the method. This implements the second of the three elementary rewrite rules for the language.

As an example of transposition, below we swap the right input of an AND gate with its output, obtaining a new relation:

In [28]:
and_transp = and_.transpose([0, 2, 1])
print(f"{sorted(and_.to_set())       = }")
print(f"{sorted(and_transp.to_set()) = }")

sorted(and_.to_set())       = [(0, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 1)]
sorted(and_transp.to_set()) = [(0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 1, 1)]


As another example, we verify that the XOR gate is invariant under such permutations:

In [29]:
xor_.transpose([0, 2, 1]) == xor_

True

The `contract2` class method can be called on a concrete `Box` subclass, to compose two boxes of that box class according to the given wiring, using Boolean tensor contraction. This implements the third and final of the three elementary rewrite rules for the language.

As a simple example of composition, we generate the relation corresponding to the CNOT binary function:

$$
(a, b) \mapsto (a, a \oplus b)
$$

In [32]:
cnot_ = FinRel.contract2(
    bit.spider(3),
    [0, 2, 4],
    xor_,
    [4, 1, 3],
    [0, 1, 2, 3]
)
assert cnot_ == FinRel.from_callable((2,2), (2,2), lambda a, b: (a, a^b))

As another simple example of composition, we generate the relation corresponding to the following binary function:

$$
(a, b, c) \mapsto a \wedge (b \vee c)
$$

In [33]:
func = FinRel.contract2(
    or_,
    [1, 2, 3],
    and_,
    [0, 3, 4]
)
assert func == FinRel.from_callable((2,2,2), 2, lambda a, b, c: a&(b|c))

Finally, we look at multiple pairwise ontractions. This is merely for educational purposes: in practice, such examples should be contracted using wirings and the `contract` class method. We generate the relation corresponding to the following binary function:

$$
(a, b, c) \mapsto (a \vee c) \wedge (b \vee c)
$$

The first contraction involves two `or_` gates, with input $c$ (at index 2) copied into both, resulting in the following intermediate function:

$$
(a, b, c) \mapsto (a \vee c, b \vee c)
$$

The second contraction connects the two outputs of the intermediate function above into the two inputs of an `and_` gate, reuslting in our desired function.


In [39]:
_func = FinRel.contract2(
    or_,
    [0, 2, 3],
    or_,
    [1, 2, 4],
    out_wires=[0, 1, 2, 3, 4]
    #   inputs ^^^^^^^  ^^^^ outputs
)
assert _func == FinRel.from_callable((2,2,2), (2,2), lambda a, b, c: ((a|c), (b|c)))
func = FinRel.contract2(
    _func,
    [0, 1, 2, 3, 4],
    and_,
    [3, 4, 5]
)
assert func == FinRel.from_callable((2,2,2), 2, lambda a, b, c: (a|c)&(b|c))